<a href="https://colab.research.google.com/github/cruciatus89/Sentiment-Analysis-of-Amazon-Product-Reviews/blob/main/Sentiment_Analysis_of_Amazon_Product_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyprind
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
from torchtext import data
import torchtext
import torch.nn.functional as F
import torch.nn as nn
import pyprind
#%matplotlib inline  
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
is_cuda = torch.cuda.is_available()
print("Cuda Status on system is {}".format(is_cuda))
print(torch.cuda.get_device_name())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Preparation  of Dataset


In [ ]:
train_df = pd.read_csv("/content/drive/My Drive/amazon_review_train_data_fixed.csv")
train_df=train_df[:int(train_df.shape[0]/10)]
test_df = pd.read_csv("/content/drive/My Drive/amazon_review_test_data_fixed.csv")
test_df=test_df[:int(test_df.shape[0]/10)]
print(train_df.shape)
print(test_df.shape)
train_df=train_df.dropna()
test_df=test_df.dropna()
print(train_df.shape)
print(test_df.shape)
X=train_df['Review_text']
y=train_df['numeric_label']


In [ ]:
dummy_text = "Review of Transformers:Dark of the Moon: This is an enjoyable science fiction actioner, with a fairly plausible plot for this type movie. It also has excellent acting, both the live actors and the voice acting for the Transformer robots. Only beef is that it's language makes it inappropriate for tweens and pre-teens. Overall, excellent for adults and recommended."
dummy_label= "2"

Splitting the Test Train Dataset

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train,X_valid,y_train,y_valid=train_test_split(X, y, test_size=0.20, random_state=42)

print(f'Size of Train set {X_train.shape}\nSize of Test set {test_df.shape}\nSize of valid set {X_valid.shape}\n')

In [ ]:
train = pd.concat([X_train, y_train], axis=1)
print(train.shape)

valid = pd.concat([X_valid, y_valid], axis=1)
print(valid.shape)

In [ ]:
train.to_csv("/content/drive/My Drive/train.csv", index=False)
test_df.to_csv("/content/drive/My Drive/test.csv", index=False)
valid.to_csv("/content/drive/My Drive/valid.csv", index=False)

In [ ]:
del train_df,train, test_df, valid, X_train, y_train,X_valid,y_valid

Tokenizing Dataset and using Pytorchtext


In [ ]:
def tokenizer(text):
    return [tok for tok in nltk.word_tokenize(text)]

In [ ]:
TEXT = data.Field(sequential=True, tokenize=tokenizer,include_lengths = True)
LABEL = data.LabelField(dtype=torch.float, sequential=False)

In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path="/content/drive/My Drive/", train="train.csv", 
    validation="valid.csv", test="test.csv",format="csv", skip_header=True, 
    fields=[('Review_text', TEXT), ('numeric_Label', LABEL)]
)

In [ ]:
TEXT.build_vocab(train_data, vectors=torchtext.vocab.Vectors("/content/drive/My Drive/glove.840B.300d.txt"), 
                 max_size=50000)
LABEL.build_vocab(train_data)

In [ ]:
# Hyperparameters
num_epochs = 30
learning_rate = 1e-4

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
OUTPUT_DIM = 1
#N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # padding
BATCH_SIZE = 8

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# keep in mind the sort_key option 
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), sort_key=lambda x: len(x.Review_text),sort_within_batch = True,
    batch_size=BATCH_SIZE,
    device=device)

In [ ]:
class LSTM_net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num directions]
        # output over padding tokens are zero tensors
        
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]
        
        # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        output = self.fc1(hidden)
        output = self.dropout(self.fc2(output))
                
        #hidden = [batch size, hid dim * num directions]
            
        return output

In [ ]:
#creating instance of our LSTM_net class

model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,  
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
#  to initiaise padded to zeros
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

In [ ]:
model.to(device) #CNN to GPU


# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
# training function 
def train(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    bar = pyprind.ProgBar(len(iterator), bar_char='█')
    for batch in iterator:
        
        text, text_lengths = batch.Review_text
        
        optimizer.zero_grad()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.numeric_Label)
        acc = binary_accuracy(predictions, batch.numeric_Label)

        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        bar.update()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator,mode):
    
    epoch_acc = 0
    epoch_loss=0
    preds,labels=[],[]
    model.eval()
    
    with torch.no_grad():
        bar = pyprind.ProgBar(len(iterator), bar_char='█')
        for batch in iterator:
            
            text, text_lengths = batch.Review_text
            predictions = model(text, text_lengths).squeeze(1)
            acc = binary_accuracy(predictions, batch.numeric_Label)
            loss = criterion(predictions, batch.numeric_Label)
            
            rounded_preds = torch.round(torch.sigmoid(predictions))
            logits = rounded_preds.detach().cpu().numpy()
            label_ids = batch.numeric_Label.to('cpu').numpy()
            
            preds.append(logits)
            labels.append(label_ids)

            epoch_acc += acc.item()
            epoch_loss+=loss.item()
            bar.update()
    if mode =="validation":        
      return epoch_acc / len(iterator),epoch_loss / len(iterator)
    if mode =="testing":
      return epoch_acc / len(iterator),epoch_loss / len(iterator),preds,labels

In [ ]:
t = time.time()
loss=[]
acc=[]
val_acc=[]
val_loss=[]
same_loss_count=0
prev_loss = 0
for epoch in range(num_epochs):
    print(f'Epoch Number: {epoch}\n')
    train_loss, train_acc = train(model, train_iterator)
    valid_acc,valid_loss = evaluate(model, valid_iterator,"validation")

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
    
    #print(f'\t Val. Acc: {valid_acc*100:.2f}%')
    
    loss.append(train_loss)
    acc.append(train_acc)
    val_acc.append(valid_acc)
    val_loss.append(valid_loss)
    
    if np.abs(prev_loss-valid_loss)<0.001:
      same_loss_count+=1

    prev_loss=valid_loss
    if same_loss_count == 3:
      break
      
print(f'time:{time.time()-t:.3f}')

In [ ]:
plt.xlabel("runs")
plt.ylabel("normalised measure of loss/accuracy")
x_len=list(range(len(val_loss)))
plt.axis([0, max(x_len), 0, 1])
plt.title('result of LSTM')
loss=np.asarray(loss)/max(loss)
plt.plot(x_len, loss, 'r+',label="loss")
plt.plot(x_len, acc, 'b.', label="accuracy")
plt.plot(x_len, val_acc, 'g.', label="val_accuracy")
plt.plot(x_len, val_loss, 'g+', label="val_loss")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.2)
plt.show

In [ ]:
test_acc,test_loss,preds,labels = evaluate(model, test_iterator,"testing")

In [ ]:
print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
preds = np.concatenate(preds)
labels = np.concatenate(labels)

prc,rec,fs = precision_score(labels,preds),recall_score(labels,preds),f1_score(labels,preds)

In [ ]:
print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Precision: {prc*100:.2f}% | Recall: {rec*100:.2f}% | F1-Score: {fs*100:.2f}%')